In [0]:
  from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once in a notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once in a notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
listed = drive.ListFile({'q': "title contains '.tsv'"}).GetList()
for file in listed:
  print('title {}, id {}'.format(file['title'], file['id']))

title multilingual.tsv, id 1G9L_GIt7NKKwsukgrSm-ZwdGReFZg9-v
title multilingual.tsv, id 1OishOd0Ma2tfSVP3RlLicUlq8rQLQFt2
title multilingual.tsv, id 1Tv4W0dlYEzVoPRWJeGrwCUtuE-1Q3R1K
title amazon_reviews_multilingual_DE_v1_00.tsv, id 1dEO1ofXilgv_mMUwH6bC5F2HFnGuExaO


In [0]:
#DOWNLOADING title amazon_reviews_multilingual_DE_v1_00.tsv TO COLAB
import pandas as pd

file_id = '1dEO1ofXilgv_mMUwH6bC5F2HFnGuExaO'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('multilingual.tsv')
df = pd.read_csv('multilingual.tsv', sep='\t', error_bad_lines=False)
print('Success!')

Success!


In [0]:
df.keys()


Index(['marketplace', 'customer_id', 'review_id', 'product_id',
       'product_parent', 'product_title', 'product_category', 'star_rating',
       'helpful_votes', 'total_votes', 'vine', 'verified_purchase',
       'review_headline', 'review_body', 'review_date'],
      dtype='object')

In [0]:
df.head()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,DE,10133,RVOG49N0H1FB6,B004TACMZ8,569741360,Bosch GMS120 Ortungsgerät digital multi-Scanner,Home Improvement,5,0,0,N,Y,Super,Delivery took a little bit more then i expecte...,2014-08-01
1,DE,19612,RNCMD6OLTP4HM,1846071224,785505948,The Wheels On The Bus: Favourite Nursery Rhyme...,Books,5,1,1,N,Y,Great compilation,We enjoy listening to the song as preparation ...,2014-12-04
2,DE,19612,R4AUOBI8YC0R8,0375851569,516548029,Dr. Seuss's Beginner Book Collection,Books,5,0,0,N,Y,Great Collection,Very great compilation. Interesting story and ...,2014-12-04
3,DE,19677,R1VSHIJ1RHIBTE,B0060SVG54,302116447,Zwei an einem Tag,Video DVD,5,0,0,N,Y,Guter Verfilmung,Den Film habe ich bereits vor lesen des Buches...,2015-07-16
4,DE,19999,R3JBLVALWSLCZD,B00EYQ6CVC,368843515,"Dr. House - Die komplette Serie, Season 1-8 (L...",Video DVD,5,9,14,N,Y,Kauft diese Box!,"Die Box ist super verarbeitet, sieht gut aus b...",2014-02-08


In [0]:
df.columns

Index(['marketplace', 'customer_id', 'review_id', 'product_id',
       'product_parent', 'product_title', 'product_category', 'star_rating',
       'helpful_votes', 'total_votes', 'vine', 'verified_purchase',
       'review_headline', 'review_body', 'review_date'],
      dtype='object')

In [0]:
type(df.review_body)
#df.at[1,'review_body']

pandas.core.series.Series

In [0]:
df.columns = ['marketplace', 'customer_id', 'review_id', 'product_id','product_parent', 'product_title', 'product_category', 'star_rating','helpful_votes', 'total_votes', 'vine', 'verified_purchase','review_headline', 'review_body', 'review_date']
print("Success")

Success


In [0]:
train1 = pd.DataFrame(pd.concat([df['customer_id'], df['review_id'],df['product_id'],df['product_title'],df['product_category'],df['review_headline'],df['review_body']], axis=1))
train1.head()

,customer_id,review_id,product_id,product_title,product_category,review_headline,review_body
0,10133,RVOG49N0H1FB6,B004TACMZ8,Bosch GMS120 Ortungsgerät digital multi-Scanner,Home Improvement,Super,Delivery took a little bit more then i expecte...
1,19612,RNCMD6OLTP4HM,1846071224,The Wheels On The Bus: Favourite Nursery Rhyme...,Books,Great compilation,We enjoy listening to the song as preparation ...
2,19612,R4AUOBI8YC0R8,0375851569,Dr. Seuss's Beginner Book Collection,Books,Great Collection,Very great compilation. Interesting story and ...
3,19677,R1VSHIJ1RHIBTE,B0060SVG54,Zwei an einem Tag,Video DVD,Guter Verfilmung,Den Film habe ich bereits vor lesen des Buches...
4,19999,R3JBLVALWSLCZD,B00EYQ6CVC,"Dr. House - Die komplette Serie, Season 1-8 (L...",Video DVD,Kauft diese Box!,"Die Box ist super verarbeitet, sieht gut aus b..."


In [0]:
train1.dtypes

customer_id          int64
review_id           object
product_id          object
product_title       object
product_category    object
review_headline     object
review_body         object
dtype: object

In [0]:
import numpy as np
import nltk
nltk.download('stopwords')
nltk.download('punkt')
import re  
from nltk.corpus import stopwords
stops1 = set(stopwords.words("english"))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [0]:
def clean_sent(sent):
    sent = sent.lower()
    sent = re.sub(u'[_"\-;%()|+&=*%.,!?:#$@\[\]/]',' ',sent)
    return sent
def clean(df):
    df['review_body'] = df.review_body.map(lambda x: ' '.join([ word for word in
                                                         nltk.word_tokenize(clean_sent(x))]).encode('utf-8'))

    df['product_title'] = df.product_title.map(lambda x: ' '.join([ word for word in
                                                         nltk.word_tokenize(clean_sent(x))]).encode('utf-8'))
def removeStopWords(df, stop):
	df['review_body'] = df.review_body.map(lambda x: ' '.join([word for word in nltk.word_tokenize(x.decode('utf-8'))
                                                         if word not in stop]).encode('utf-8'))
	df['product_title'] = df.product_title.map(lambda x: ' '.join([word for word in nltk.word_tokenize(x.decode('utf-8'))
                                                         if word not in stop]).encode('utf-8'))


In [0]:
#Calling Clean and Stop words on Amazon Product data DF
clean(train1)
removeStopWords(train1, stops1)

In [0]:
#Checking for data after Pre-processing
train1.at[1,'review_body']

b'enjoy listening song preparation baby songs quite new even us hope baby enjoy songs much'

In [0]:
train1.keys()

Index(['customer_id', 'review_id', 'product_id', 'product_title',
       'product_category', 'review_headline', 'review_body'],
      dtype='object')

In [0]:
#Statistics after Data Preprocessing
train1.replace('', np.nan, inplace=True)
dirty_data = train1[train1.isnull().any(axis=1)]
print ('dirty sample count:', dirty_data.shape[0])
print ('positive dirty training sample:', len(dirty_data[dirty_data['product_category'] == 1]))
print ('negative dirty training sample:', len(dirty_data[dirty_data['product_category'] == 0]))

train1 = train1.dropna()
    
print ('Train sample count:', train1.shape[0])
train1.columns = ['customer_id', 'review_id', 'product_id', 'product_title',
       'product_category', 'review_headline', 'review_body']

dirty sample count: 1
positive dirty training sample: 0
negative dirty training sample: 0
Train sample count: 678992
